In [1]:
# switch to the project directory
%cd ..
# working directory should be ../pdi

c:\Users\admin\Desktop\research\pdi


In [2]:
import sys
import os
module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pdi.constants import (
    PARTICLES_DICT,
    TARGET_CODES
)
from pdi.data.constants import GROUP_ID_KEY

In [4]:
import torch
import torch.nn as nn
from pdi.data.preparation import FeatureSetPreparation
from pdi.models import AttentionModel
from pdi.data.types import Split
import onnx
import onnxruntime as ort
import json

device = torch.device("cpu")


data_preparation = FeatureSetPreparation()
(train_loader, ) = data_preparation.prepare_dataloaders(64, 0, [Split.TRAIN])

it = iter(train_loader)
dummy_input1, _, _ = next(it)
dummy_input2, _, _ = next(it)

dummy_input1.to(device)
dummy_input2.to(device)

thresholds = {}

os.makedirs(f"onnx/Proposed/", exist_ok=True)
for target_code in TARGET_CODES:
    load_path = f"models/Proposed/{PARTICLES_DICT[target_code]}.pt"
    export_path = f"onnx/Proposed/{PARTICLES_DICT[target_code]}.onnx"
    saved_model = torch.load(load_path)
    model = AttentionModel(*saved_model["model_args"]).to(device)
    model.eval()
    model.thres = saved_model["model_thres"]
    model.load_state_dict(saved_model["state_dict"])

    thresholds[PARTICLES_DICT[target_code]] = float(model.thres)

    torch.onnx.export(model, (dummy_input1, dummy_input2), export_path, input_names=["input"], output_names=["output"], 
        dynamic_axes={'input' : {0 : 'batch_size', 1 : 'feature_num'},    # variable length axes
                      'output' : {0 : 'batch_size'}})

    onnx_model = onnx.load(export_path)
    onnx.checker.check_model(onnx_model)


    # ort_sess = ort.InferenceSession(export_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    # outputs1 = ort_sess.run(None, {'input': dummy_input1.numpy()})
    # outputs2 = ort_sess.run(None, {'input': dummy_input2.numpy()})

    # predicted1 = model(dummy_input1)
    # predicted2 = model(dummy_input2)

    # # Print Result 


    # print(f"Difference1: {predicted1.detach().numpy() - outputs1[0]}")
    # print(f"Difference2: {predicted2.detach().numpy() - outputs2[0]}")

    # print(dummy_input1.size())
    # print(dummy_input2.size())
    # break
with open("onnx/thresholds.json", "w") as fp:
    json.dump(thresholds, fp)